In [1]:
%config IPCompleter.evaluation='unsafe'

In [2]:
from pyiron_workflow import Workflow, as_function_node   

import pyiron_nodes as pn
from pyironflow.pyironflow import PyironFlow

In [3]:
pn.atomistic.calculator.data.OutputCalcMD.__qualname__, pn.atomistic.calculator.data.OutputCalcMD.__module__

('OutputCalcMD', 'pyiron_nodes.atomistic.calculator.data')

In [4]:
pn.atomistic.calculator.data.OutputCalcMD.dataclass()

OutputCalcMD(energies_pot=None, energies_kin=None, forces=None, positions=None, temperatures=None)

In [5]:
from pyironflow.reactflow import get_import_path

In [6]:
md_out_node = pn.atomistic.calculator.data.OutputCalcMD()

print (md_out_node.outputs.channel_dict.energies_kin)
get_import_path(md_out_node)
# md_out_node.inputs
# md_out_node.__module__

NOT_DATA


'pyiron_nodes.atomistic.calculator.data.OutputCalcMD'

In [7]:
inp_node = pn.atomistic.calculator.data.InputCalcMD()
get_import_path(inp_node)
inp_node.__module__

'pyiron_nodes.atomistic.calculator.data'

In [8]:
md_out_node.as_path

<bound method Semantic.as_path of <pyiron_nodes.atomistic.calculator.data.OutputCalcMD object at 0x103dcf860>>

In [9]:
import tempfile
import os
from importlib.util import spec_from_file_location, module_from_spec

import typing
import dataclasses
from dataclasses import dataclass, field


def generate_code(dc: dataclasses.dataclass):
    name = dc.__class__.__name__
    fields = [f.name for f in dataclasses.fields(dc)]
    fields_with_type = [f"{f.name}: {f.type.__name__}" for f in dataclasses.fields(dc)]
    fields_code = '\n    '.join([f'{field} = dc.{field}' for field in fields])
    return_code = ', '.join(fields)
    return f'''
from dataclasses import dataclass
from pyiron_workflow import as_function_node

@as_function_node
def {name}(dc: dataclass):
    {'\n    '.join(fields_with_type)}
    {fields_code}
    return {return_code}
    '''

In [10]:
def as_output_node(data):
    dc = dataclass(data)
    # Generate code
    code = generate_code(dc())
    # print (code)

    # Create a temporary file
    temp_dir = tempfile.gettempdir()
    file_name = os.path.join(temp_dir, 'temp_file.py')
    
    # Write code to file
    with open(file_name, 'w') as temp_file:
        temp_file.write(code)
        
    # Import the function from the temp file
    spec = spec_from_file_location('temp_module', file_name)
    module = module_from_spec(spec)
    spec.loader.exec_module(module)

    node = getattr(module, dc.__name__)
    node.dataclass = dc
    
    # Delete the temporary file when done
    os.remove(file_name)    
    
    return node

In [11]:
from dataclasses import field
from typing import Optional

@as_output_node
class OutputCalcMD:
    import numpy as np

    energies_pot: Optional[np.ndarray] = field(
        default=None
    )
    energies_kin: Optional[np.ndarray] = field(
        default=None
    )
    forces: Optional[np.ndarray] = field(default=None)
    positions: Optional[np.ndarray] = field(
        default=None
    )
    temperatures: Optional[np.ndarray] = field(
        default=None
    )

In [12]:
%%time

node = OutputCalcMD()
node.dataclass

CPU times: user 126 μs, sys: 0 ns, total: 126 μs
Wall time: 128 μs


__main__.OutputCalcMD

In [13]:
xx

NameError: name 'xx' is not defined

In [ ]:
from pyiron_nodes.atomistic.calculator.data import OutputCalcMD

node = as_output_node(OutputCalcMD)
node().node_function

In [ ]:
node(OutputCalcMD()).run()

In [ ]:
xx

### A few test constructs to convert a dataclass into a function node

In [ ]:
from pyiron_workflow import Workflow, as_function_node 

In [ ]:
from dataclasses import dataclass
from pyiron_workflow import as_function_node

@as_function_node
def OutputCalcMD(dc: dataclass):
    energies_pot: Optional
    energies_kin: Optional
    forces: Optional
    positions: Optional
    temperatures: Optional
    energies_pot = dc.energies_pot
    energies_kin = dc.energies_kin
    forces = dc.forces
    positions = dc.positions
    temperatures = dc.temperatures
    return energies_pot, energies_kin, forces, positions, temperatures

In [ ]:
OutputCalcMD()

In [ ]:
pf = PyironFlow([])
pf.gui

In [ ]:
wf = pf.get_workflow()

In [ ]:
t = wf.InputCalcMD.inputs.temperature
t

In [ ]:
t2 = wf.SetInputCalcMD.inputs.temperature
check_type(t2.value, t2.type_hint), type(t2.type_hint)

In [ ]:
from typeguard import check_type
from typing import Optional

check_type(t.value, eval(t.type_hint)), type(t.type_hint)

In [ ]:
from pyiron_workflow import Workflow, as_dataclass_node, as_function_node, as_macro_node

In [ ]:
from pyiron_nodes.atomistic.calculator.data import InputCalcMD

wf = Workflow('test')
wf.input = InputCalcMD()

type(wf.input.inputs.temperature.type_hint)

In [ ]:
from __future__ import annotations

from dataclasses import field
from typing import Optional

from pyiron_nodes.dev_tools import wf_data_class, wfMetaData
from pyiron_workflow import as_dataclass_node

from pyiron_workflow import Workflow

@as_dataclass_node
class InputCalcMD:
    temperature: Optional[int | float] = 300
    n_ionic_steps: int = 10_000
    n_print: int = 100
    pressure: Optional[int | float] = None
    time_step: Optional[int | float] = 1.0
    temperature_damping_timescale: Optional[int | float] = 100.0
    pressure_damping_timescale: Optional[int | float] = 1000.0
    seed: Optional[int] = None
    tloop: Optional[float] = None
    initial_temperature: Optional[float] = None
    langevin: bool = False
    delta_temp: Optional[float] = None
    delta_press: Optional[float] = None

type(InputCalcMD().inputs.temperature.type_hint)

In [ ]:
@as_dataclass_node
class MyData:
    test: int


type(MyData().inputs.test.type_hint)

In [ ]:
type(int)

In [ ]:
from pyiron_nodes.dev_tools import wf_data_class

@wf_data_class()
#@as_dataclass_node
class my_data:
    test: int
    test2: int = 10

In [ ]:
my_data(test=10)

In [ ]:
@as_function_node
def my_func(data):
    test = data.test
    return test

In [ ]:
mf = my_func(my_data(test=1))

In [ ]:
wf = Workflow('test')

wf.data = my_data(test=10)
wf.f1 = my_func(data = wf.data)


In [ ]:
wf.data

In [ ]:
wf.run()

In [ ]:
import numpy as np
np.arange(3)

In [ ]:
@as_function_node('my_range')
def get_list(max_num: int = 3):
    return np.arange(max_num).tolist()

type(get_list().run()[0])

In [ ]:
@as_macro_node('f1')
def func_macro(wf, test=20, max_num=5):
    wf.data = my_data(test=test)
    wf.f1 = my_func(data = wf.data)
    wf.range = get_list(max_num=wf.data.outputs.dataclass.test2)
    wf.loop = wf.create.for_node(
        body_node_class = my_data,
        iter_on = ('test',),
        test = wf.range,
    )
    return wf.loop

In [ ]:
my_data(10)

In [ ]:
@as_macro_node('f1')
def func_macro(wf, test=20, max_num=5, data: my_data = my_data(10)):
    wf.f1 = my_func(**my_data)
    # wf.range = get_list(max_num=data.outputs.dataclass.test2)
    # wf.loop = wf.create.for_node(
    #     body_node_class = my_data,
    #     iter_on = ('test',),
    #     test = wf.range,
    # )
    return wf.f1 # wf.loop

In [ ]:
fm = func_macro()
fm.run()

In [ ]:
func_macro()

In [ ]:
from pyiron import Project

In [ ]:
pr = Project('test')
Al = pr.create.structure.bulk('Al', cubic=True)
job = pr.create.job.Lammps('test')
job.structure = Al
job.run(delete_existing_job=True)

In [ ]:
from pyiron_workflow import Workflow


@Workflow.wrap.as_function_node
def Report(t1, t2, t3):
    tmax = max(t1, t2, t3)
    print("LONGEST", tmax)
    return tmax

wf = Workflow("sleepy")
wf.t_sleep = Workflow.create.standard.UserInput(1)
wf.a1 = Workflow.create.standard.Sleep(wf.t_sleep)
wf.a2 = Workflow.create.standard.Sleep(wf.t_sleep)
wf.a3 = Workflow.create.standard.Sleep(wf.t_sleep)
wf.midway = Report(wf.a1, wf.a2, wf.a3)
wf.b1 = Workflow.create.standard.Sleep(wf.midway)
wf.b2 = Workflow.create.standard.Sleep(wf.midway)
wf.b3 = Workflow.create.standard.Sleep(wf.midway)
wf.end = Report(wf.b1, wf.b2, wf.b3)

from time import time

t0 = time()

with Workflow.create.ThreadPoolExecutor(max_workers=3) as exe:
    for n in wf:
        if n.label not in ["t_sleep", "midway", "finally"]:
            n.executor = exe
    wf()
    
print("Total runtime", time() - t0)

In [ ]:
from executorlib import Executor

with Executor(max_cores=1, backend="local") as exe:
    for i in range(3):
        future = exe.submit(sum, [1, i], resource_dict={"cores": 1})
        print(future.result())

In [ ]:
from pyiron import Project

In [ ]:
pr = Project('test')
Re = pr.create.structure.bulk('Re')
job = pr.create.job.Lammps('lammps')
job.structure = Re
job.list_potentials()